# Modulos

In [ ]:
import os
import re
import zipfile
from pathlib import Path
from datetime import datetime, timedelta
import pandas as pd
import io


# Rutas y variables

In [ ]:


# =============================================================================
# CONFIGURACIÓN Y RUTAS
# =============================================================================
ruta_base = r"\\Servernas\AYC2\(01. ASEGURAMIENTO)\01. ASEGURAMIENTO\01. REGIMEN SUBSIDIADO\MUNICIPIOS 2025\LMA_CONTRIBUCION_SOLIDARIA\CS LMA"

datos_lista = []
archivos_procesados = []
archivos_error = []

# =============================================================================
# PROCESAMIENTO DE ARCHIVOS
# =============================================================================
for root, dirs, files in os.walk(ruta_base):
    for file in files:
        if file.endswith('.zip'):
            zip_path = os.path.join(root, file)
            print(f"Procesando ZIP: {zip_path}")
            
            try:
                with zipfile.ZipFile(zip_path, 'r') as zip_ref:
                    # Filtrar solo archivos .txt dentro del ZIP
                    txt_files = [f for f in zip_ref.namelist() if f.endswith('.txt')]
                    
                    for txt_file in txt_files:
                        try:
                            # 1. Extracción de metadatos del nombre del archivo
                            nombre_sin_ext = txt_file.split('/')[-1].replace('.txt', '')
                            fecha_str = nombre_sin_ext[-8:]  # ddmmyyyy
                            fecha_formateada = datetime.strptime(fecha_str, '%d%m%Y').strftime('%d/%m/%Y')
                            
                            # 2. Lectura del contenido sin encabezados
                            with zip_ref.open(txt_file) as f:
                                # Aplicamos header=None para que la primera fila se trate como datos
                                df_temp = pd.read_csv(
                                    f, 
                                    sep='|', 
                                    encoding='utf-16-le', 
                                    dtype=str, 
                                    header=None  # <--- Crucial: evita la pérdida de la primera fila
                                )
                                
                                # 3. Enriquecimiento del DataFrame
                                df_temp['ARCHIVO_ORIGEN'] = txt_file
                                df_temp['FECHA_ARCHIVO'] = fecha_formateada
                                
                                datos_lista.append(df_temp)
                                archivos_procesados.append({
                                    'archivo': txt_file, 
                                    'fecha': fecha_formateada, 
                                    'registros': len(df_temp)
                                })
                                
                                print(f"  ✓ Procesado: {txt_file} | Registros: {len(df_temp)}")
                                
                        except Exception as e:
                            archivos_error.append({'archivo': txt_file, 'error': str(e)})
                            print(f"  ✗ Error en contenido de {txt_file}: {str(e)}")
                            
            except Exception as e:
                print(f"  ✗ Error crítico al abrir ZIP {file}: {str(e)}")

# =============================================================================
# CONSOLIDACIÓN DE RESULTADOS
# =============================================================================
if datos_lista:
    # Unificamos todos los fragmentos
    df = pd.concat(datos_lista, ignore_index=True)
    
    # Si conoces los nombres de las columnas, puedes asignarlos aquí (opcional)
    # df.columns = ['Col1', 'Col2', ..., 'ARCHIVO_ORIGEN', 'FECHA_ARCHIVO']
    
    print(f"\n{'='*60}")
    print(f"RESUMEN DE PROCESAMIENTO FINAL")
    print(f"{'='*60}")
    print(f"Total archivos procesados: {len(archivos_procesados)}")
    print(f"Total archivos con error:  {len(archivos_error)}")
    print(f"Total registros finales:   {len(df)}")
    print(f"\nDistribución por fecha de archivo:")
    # Nota: Como no hay encabezados, la columna de fecha será la última o penúltima
    print(df['FECHA_ARCHIVO'].value_counts().sort_index())
else:
    print("\n[!] No se consolidó ningún dato. Verifique las rutas o el formato de los archivos.")
    df = pd.DataFrame()

# dataframe

In [ ]:
print(df)

In [ ]:
df = df[df[19] == 'PAGADO']
print(f"Registros con estado PAGADO: {len(df)}")
print(df)

In [ ]:
# Mostrar cantidad de registros inicial
print(f"Registros iniciales en df: {len(df)}")
print(f"\nEjemplo de duplicados (mismo registro y periodo):")
print(df.groupby([3, 4]).size().sort_values(ascending=False).head(10))

# Guardar cantidad inicial para comparación
registros_iniciales = len(df)

# Eliminar duplicados por columnas 3 (registro único) y 4 (periodo)
# keep='first' mantiene la primera ocurrencia
df = df.drop_duplicates(subset=[3, 4], keep='first')

# Mostrar resultados del proceso
registros_finales = len(df)
registros_eliminados = registros_iniciales - registros_finales

print(f"\n{'='*60}")
print(f"RESUMEN DE ELIMINACIÓN DE DUPLICADOS")
print(f"{'='*60}")
print(f"Registros iniciales:   {registros_iniciales:,}")
print(f"Registros finales:     {registros_finales:,}")
print(f"Registros eliminados:  {registros_eliminados:,}")
print(f"Porcentaje eliminado:  {(registros_eliminados/registros_iniciales)*100:.2f}%")

# Verificar que no quedan duplicados
duplicados_restantes = df.duplicated(subset=[3, 4]).sum()
print(f"\nDuplicados restantes (registro + periodo): {duplicados_restantes}")

In [ ]:
# Definir la ruta del archivo Excel de salida
output_path = os.path.join(ruta_base, 'df_consolidado.xlsx')

# Exportar el dataframe a Excel
df.to_excel(output_path, index=False, engine='openpyxl')

print(f"Archivo exportado exitosamente en: {output_path}")
print(f"Total de registros exportados: {len(df)}")